# TODO
* ~~Create a filter for equipments, adding to wo_type and clientid~~
* Change times, complete and incomplete
* Comment and attempt to refactor

# Read data and calculate TTF for each failure mode



In [ ]:
%%html
<script>
code_show=true;
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
To hide/show code, click <a href="javascript:code_toggle()">here</a>.


In [ ]:
import sys
sys.path.append(r'..\bin')
import pandas as pd
import numpy as np
import ipywidgets as widgets
import json
import matplotlib.pyplot as plt

from IPython.display import HTML, Image, display, clear_output
from sqlalchemy import MetaData
from sqlalchemy.sql import text

import analytics_stat.db_mgmt as db

%matplotlib inline


In [ ]:
def create_selector(wo_table):
    """
    Given a dataframe, returns a display to select from different types of workorders with dropdowns
    """
    
    wo_type_labels_nano = ['-', 'PRV', 'PRD', 'COR', 'INS', 'CLN', 'LUB', 'MET', 'LAC', 'COM', 'CON', 'GEN']
    container = list()  # Container of the VBox
    dropdown_selectors = list()  # Dropdown list
    

    for i, label in enumerate(wo_table['wo_type']):
        dropdown_selectors.append(widgets.Dropdown(options=wo_type_labels_nano, description=label, disabled=False))
        
    container.append(widgets.VBox(dropdown_selectors))
    type_selects = list()

    return container

In [ ]:
class work_orders:
    
    def __init__(self):
        
        self._reliability_table_status = None
        self._clientid = None
        self._symptom = None
        self._select_wo = None
        self._top_freq_wo = None
        self._ttf = None
        self._raw_data = None
        
        self._dropdown_client = widgets.Dropdown(value= '-', description='Client:', options=['-'],
                                                 continuous_update=True)
        self._dropdown_client.disabled = False
        self._dropdown_client.observe(self.update_clientid, 'value')
        
        self._dropdown_equipment = widgets.Dropdown(value= '-', description='Equipment:', options=['-'],
                                                    continuous_update=True)
        self._dropdown_equipment.disabled = True
        self._dropdown_equipment.observe(self.update_equipment, 'value')
        
        self._commit_button = widgets.Button(description='Update all\nWO types',
                                             layout=widgets.Layout(width='200px', 
                                                                   height='80px'))
        self._commit_button.on_click(self.update_wo_call,)

In [ ]:
def read_reliability_table_status(self):
    # Read the table from a query
    resulting_table = pd.read_sql_query("""SELECT c.id as clientid, c.name, COALESCE( r.cnt, 0 ) AS n_rows
                                    FROM clients c
                                    LEFT JOIN 
                                    (SELECT clientid, COUNT(*) AS cnt FROM reliability GROUP BY clientid) r
                                    ON r.clientid = c.id
                                    ORDER BY n_rows DESC;"""
                                    , engine)
    
    self._reliability_table_status = resulting_table
    
    display(self._reliability_table_status)
    
    # Create the options for the client selection list
    self._client_names = ['-'] + list(np.sort(
        self._reliability_table_status.loc[self._reliability_table_status['n_rows']>0,'name'].values)) 
    self._dropdown_client.options = self._client_names

In [ ]:
def iniciate_client_choise(self):
    display(self._dropdown_client)

In [ ]:
def update_clientid(self,args):
    
    clear_output() # Makes display disappear
    
    print('Selected Client:\t{}'.format(self._dropdown_client.value))
    # if the value is chosen     
    if self._dropdown_client.value != '-':
          
        self._clientid  = self._reliability_table_status.loc[self._reliability_table_status['name'] == 
                                                             self._dropdown_client.value
                                                             ,'clientid'].values[0]
        
        print('Client id:\t\t{}'.format(self._clientid))
          
        self._dropdown_equipment.disabled = False
        
        wo_header = pd.read_sql_query("""SELECT  wo_header_short, wo_type, wo_type_nano, eq_id 
                                        FROM reliability WHERE clientid = {}; """.format(self._clientid), engine)
        
        self._dropdown_equipment.options = ['-'] + list(set(wo_header['eq_id'].values))
        display(self._dropdown_equipment)
            
    else:
        self._clientid  = None
        print('Client id:\t\t{}'.format('-'))    

In [ ]:
def update_equipment(self, args):
    
    self._equipment = self._dropdown_equipment.value
    clear_output()
    print('Selected Client:\t{}'.format(self._dropdown_client.value))
    print('Client id:\t\t{}'.format(self._clientid))
    print('Equipment:\t\t{}'.format(self._equipment))
    self.config_wo(0)

In [ ]:
# def update_symptom(self,args):
#     clear_output()
#     print('Selected Symptom:\t{}'.format(args['new']))
    
#     if args['new']!='All':
#         self._symptom  = reliability_status.loc[reliability_status['name']==select_client.value, 'clientid'].values[0]
#         print('Selected symptom:\t\t{}'.format(self._clientid))
#     else:
#         self._symptom  = None
#         print('Selected symptom:\t\t{}'.format('All'))

In [ ]:
def config_wo(self,args):

    if self._clientid and self._equipment:
        wo_header = pd.read_sql_query("""SELECT  wo_header_short, wo_type, wo_type_nano 
                                      FROM reliability WHERE clientid = {} AND eq_id = '{}'; """.format(self._clientid, self._equipment)
                                      , engine)
        self._test_variable = wo_header
        if wo_header['wo_type'].count() != 0 :

            # TODO: Normalize texts to be more efficient in the group by
            self._top_freq_wo = wo_header.groupby(['wo_type','wo_header_short']).size() \
                                .to_frame('count_header').reset_index() \
                                .groupby(['wo_type','count_header'])['wo_header_short'].unique() \
                                .to_frame().reset_index() \
                                .sort_values('count_header', ascending=False) \
                                .drop_duplicates(subset='wo_type')\
                                .sort_values('wo_type')\
                                .reset_index()

            display(self._top_freq_wo.iloc[:, 1:])

            self._select_wo = widgets.HBox(create_selector(self._top_freq_wo),
            #                      layout=widgets.Layout(width='50%', height='10px')
                                 )
                
            display(self._select_wo)
            display(self._commit_button)
        else:
            print('Client has no WO types specified.')


    elif self._clientid is None:
        print('Please, select client.')

In [ ]:
def update_wo_call(self, args):
    if self._top_freq_wo is not None:
        wo_type_labels = list(self._top_freq_wo['wo_type'].values)
        query_labels = str()
        for i,label in enumerate(self._select_wo.children[0].children):
            query_labels += "WHEN wo_type = '{}' AND clientid = {} AND eq_id = '{}' THEN '{}'\n".format(label.description, self._clientid, self._equipment, label.value)

        query_update_wo = """UPDATE reliability 
               SET wo_type_nano = 
               CASE 
               {}
               ELSE wo_type_nano 
               END;""".format(query_labels)

        table_name = 'reliability'

        exists=engine.execute(text("""SELECT * FROM information_schema.tables WHERE table_name = '{}'""".format(table_name)).execution_options(autocommit=True)).fetchall()
        if exists:
            engine.execute(text(query_update_wo).execution_options(autocommit=True))
                
        print('changes were commited.')
    else:
        print('no changes.')

In [ ]:
def compute_ttf(self, args):
    if self._clientid:
        if self._symptom:
            raw_data = pd.read_sql_query("""SELECT * FROM reliability WHERE clientid = {} AND symptoms = {}; """.format(self._clientid), engine)
        elif self._symptom is None:
            raw_data = pd.read_sql_query("""SELECT * FROM reliability WHERE clientid = {} AND eq_id = '{}'; """.format(self._clientid, self._equipment), engine)

        raw_data = raw_data.sort_values(by='wo_type_nano')
        raw_data = raw_data.sort_values(by='wo_start_datetime')
        try:
            raw_data['TTR'] = (raw_data['wo_finish_datetime'] - raw_data['wo_start_datetime'])
            raw_data['TTR (Days)'] = raw_data['TTR'].astype('timedelta64[D]')

            
            raw_data['TTF'] = raw_data.loc['wo_start_datetime'].sort_values().diff() - raw_data.loc['TTR'].values

            raw_data['TTF (Days)'] = raw_data['TTF'].astype('timedelta64[D]')
        except:
            raw_data['TTF'] = raw_data['wo_start_datetime'].sort_values().diff()
            
            raw_data['TTF (Days)'] = raw_data['TTF'].astype('timedelta64[D]')  
        self._type_of_order = [1 if element == "COR" else 0 for element in raw_data['wo_type_nano'] ]        
        self._ttf = raw_data.loc[raw_data['TTF (Days)']>=0,'TTF (Days)'].values
        self._ttf  = self._ttf .ravel().astype(int)
        print(pd.DataFrame(self._ttf).describe().transpose())
        plt.hist(self._ttf)

    self.raw_data = raw_data

In [ ]:
def save_ttf(self, args):
    if self._clientid:
        if len(self._ttf)>=2:  # at lest two points
            if self._symptom:
                np.savetxt("../data/results/ttf/ttf_symptom{}_client{}.csv".format(self._symptom,self._clientid), self._ttf, delimiter=",")
            elif self._symptom is None:
                np.savetxt("../data/results/ttf/ttf_symptomAll_client{}.csv".format(self._symptom,self._clientid), self._ttf, delimiter=",")
        else:
            print('Please, compute ttf.')
        
    elif self._clientid is None:
        print('Please, select client.')

In [ ]:
def save_json(self, args):
    json_ttf = dict()
        
    if self._clientid:
        if len(self._ttf)>=2:
            if self._symptom:
                json_ttf['symptom'] = int(self._symptom)
            elif self._symptom is None:
                json_ttf['symptom'] = None
                
            json_ttf['ttf'] = self._ttf.tolist()
            json_ttf['is_failure'] = self._type_of_order
            json_ttf['client_id'] = int(self._clientid)
            json_ttf['client_name'] = self._dropdown_client.value
                                
            with open("../data/results/ttf/ttf_symptom{}_client{}.json".format(json_ttf['symptom'],
                                                                                   json_ttf['client_id']), 'w') as f:
                json.dump(json_ttf, f)
                    
            print('JSON saved.')
                
        else:
            print('Please, compute ttf.')
                
        
    elif self._clientid is None:
        print('Please, select client.')    

In [ ]:
work_orders.read_reliability_table_status = read_reliability_table_status
work_orders.iniciate_client_choise = iniciate_client_choise
work_orders.update_clientid = update_clientid
work_orders.update_equipment = update_equipment
#work_orders.update_symptom = update_symptom
work_orders.config_wo = config_wo
work_orders.update_wo_call = update_wo_call
work_orders.compute_ttf = compute_ttf
work_orders.save_ttf = save_ttf
work_orders.save_json = save_json

## Database connection

In [ ]:
engine = db.db_engine()

meta = MetaData()
meta.bind = engine
meta.reflect()

## Status of Reliability table

In [ ]:
main_hub = work_orders()
main_hub.read_reliability_table_status()

## Configure Nano's Work Orders (WO) types

In [ ]:
main_hub.iniciate_client_choise()

In [ ]:
compute_ttf = widgets.Button(description=r'Compute TTF',
                             layout=widgets.Layout(width='110px', 
                                                   height='60px'))
display(compute_ttf)
    
compute_ttf.on_click(main_hub.compute_ttf,)

In [ ]:
save_ttf = widgets.Button(description='Save TTF',
                                 layout=widgets.Layout(width='110px', 
                                                       height='60px'))
display(save_ttf)
    
save_ttf.on_click(main_hub.save_json,)